In [1]:
import sys
sys.path.append("../src")

### Load Q&A test data

In [3]:
!mc cp s3/projet-llm-insee-open-data/data/eval_data/eval_retrieval/q_and_a_scored_filtered_Phi-3-mini-128k-instruct.csv ../data/q_and_a_ref_retrieval_evaluation_Phi-3-mini-128k-instruct.csv

...struct.csv: 281.66 KiB / 281.66 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 555.67 KiB/s 0s

### Load Knowledge Data

In [9]:
!mc cp s3/projet-llm-insee-open-data/data/eval_data/eval_retrieval/insee_documents_sample_ref_retrieval_evaluation.csv ../data/insee_documents_sample_ref_retrieval_evaluation.csv

...uation.csv: 4.69 MiB / 4.69 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 8.54 MiB/s 0s

In [2]:
import pandas as pd 

#question and answer
path_qa = "../data/q_and_a_ref_retrieval_evaluation_Phi-3-mini-128k-instruct.csv"
test = pd.read_csv(path_qa)

df_dict = {}
df_dict["the_df_dataset"] = test

#knowledge data 
path_knowledge = "../data/insee_documents_sample_ref_retrieval_evaluation.csv"
data = pd.read_csv(path_knowledge)

In [ ]:
display(test.head())
display(data.head())

In [ ]:

# Define retrieval configurations
"""
RetrievalConfiguration(
    name='test1',
    database="chromadb",
    database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
    collection="test_data",
    embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
    reranker_type='BM25',
    reranker_name=None,
    k_values=[1, 2, 3, 5, 10, 20, 30, 50]
)
RetrievalConfiguration(
    name='test1',
    database="chromadb",
    database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
    collection="test_data",
    embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
    reranker_type=None,
    reranker_name=None,
    k_values=[1, 2, 3, 5, 10, 20, 30, 50]
)
RetrievalConfiguration(
    name='test1',
    database="chromadb",
    database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
    collection="test_data",
    embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
    reranker_type="Cross-encoder",
    reranker_name="BAAI/bge-reranker-base",
    k_values=[1, 2, 3, 5, 10, 20, 30, 50]
    )
RetrievalConfiguration(
            name='test1',
            database="chromadb",
            database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
            collection="test_data",
            embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
            reranker_type=None,
            reranker_name=None,
            k_values=[1, 2, 3, 5, 10, 20, 30, 50]
RetrievalConfiguration(
        name='ColBERT',
        database="chromadb",
        collection="Solon-embeddings-large-0.1",
        database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
        embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
        reranker_type="ColBERT",
        reranker_name="antoinelouis/colbertv2-camembert-L4-mmarcoFR",
        k_values=[1, 2, 3, 5, 10, 20, 30, 50]
    )
 RetrievalConfiguration(
        name='ensemble',
        database="chromadb",
        collection="data_test",
        database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
        embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
        reranker_type="Ensemble",
        param_ensemble = [
            {"reranker_type":"Cross-encoder",
             "reranker_name":"BAAI/bge-reranker-base",
             "reranker_weight": 1/4
            },
            {"reranker_type":"BM25",
             "reranker_name": None,
             "reranker_weight": 1/4
            },
            {"reranker_type":"ColBERT",
             "reranker_name": "antoinelouis/colbertv2-camembert-L4-mmarcoFR",
             "reranker_weight": 1/4
            },
            {"reranker_type":"Metadata",
             "reranker_name": "title",
             "reranker_weight": 1/4
            },
        ],
        k_values=[1, 2, 3, 5, 10, 20, 30, 50],
        rerank_k= 50
    )
"""

# Run the evaluator
results = RetrievalEvaluator.run(
    eval_configurations=eval_configs,
    eval_dict=df_dict,  # Ensure 'df_dict' is a dictionary containing pandas DataFrames with the required structure
)


In [5]:
eval_configs

[RetrievalConfiguration(name='ensemble', database='chromadb', database_path='../data/insee_documents_sample_ref_retrieval_evaluation.csv', embedding_model_name='OrdalieTech/Solon-embeddings-large-0.1', collection='data_test', chunk_size=2000, overlap_size=500, reranker_type='Ensemble', reranker_name=None, param_ensemble=[{'reranker_type': 'Cross-encoder', 'reranker_name': 'BAAI/bge-reranker-base', 'reranker_weight': 0.25}, {'reranker_type': 'BM25', 'reranker_name': None, 'reranker_weight': 0.25}, {'reranker_type': 'ColBERT', 'reranker_name': 'antoinelouis/colbertv2-camembert-L4-mmarcoFR', 'reranker_weight': 0.25}, {'reranker_type': 'Metadata', 'reranker_name': 'title', 'reranker_weight': 0.25}], use_metadata=None, rerank_k=50, k_values=[1, 2, 3, 5, 10, 20, 30, 50], markdown_separator=['\n\n', '\n', '.', ' ', ''])]

## Test 03/07/2024 - Update Retrieval Evaluation process

### Test Cross-encoder

In [ ]:
from evaluation import RetrievalConfiguration, RetrievalEvaluator

cross_encoders = ["dangvantuan/CrossEncoder-camembert-large","antoinelouis/crossencoder-electra-base-french-mmarcoFR","BAAI/bge-reranker-base"]

eval_configs = [RetrievalConfiguration(
        name=f'test_cross_encoder_{i}',
        database="chromadb",
        collection="Solon-embeddings-large-0.1_512_51",
        database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
        embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
        reranker_type="Cross-encoder",
        reranker_name=cross_encoder,
        rerank_k=50,
        k_values=[1, 2, 3, 5, 10, 20, 30, 50]
    ) for i,cross_encoder in enumerate(cross_encoders)]

# Run the evaluator
results = RetrievalEvaluator.run(
    eval_configurations=eval_configs,
    eval_dict=df_dict,  # Ensure 'df_dict' is a dictionary containing pandas DataFrames with the required structure
)

In [ ]:

for config, result in zip(eval_configs, results["the_df_dataset"].values()):
    print(config.reranker_name)
    print(result)